In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
donors = ScrapedContribution.objects.values('donor', 'city', 'state', 'employer', 'occupation').annotate(sum=Sum('amount'), num=Count('date')).order_by('-sum', '-num')
print donors.count()

228634


In [3]:
print donors[9999]

{'city': u'SEATTLE', 'sum': Decimal('2000.000'), 'employer': u'BURKE MUSEUM', 'state': u'WA', 'num': 1, 'donor': u'FERGUSON ELLEN L.', 'occupation': u'COMMUNITY RELATIONS'}


In [4]:
from csv import DictWriter

with open('/home/aepton/code/campfin/data/donors_for_dedupe.csv', 'w+') as fh:
    writer = DictWriter(fh, donors[0].keys())
    writer.writeheader()
    writer.writerows([row for row in donors[:10000]])

IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/donors_for_dedupe.csv'

In [5]:
from dedupe import StaticDedupe

with open('/home/aepton/code/campfin/data/donors.dedupe_settings') as fh:
    deduper = StaticDedupe(fh)

IOError: [Errno 2] No such file or directory: '/home/aepton/code/campfin/data/donors.dedupe_settings'

In [6]:
variables = [
    {'field' : 'donor', 'type': 'String'},
    {'field' : 'city', 'type': 'String', 'has missing':True},
    {'field' : 'state', 'type': 'String', 'has missing':True},
    {'field' : 'employer', 'type': 'String', 'has missing':True},
    {'field' : 'occupation', 'type': 'String', 'has missing':True},
]

In [7]:
from dedupe import Dedupe

data_dictionary = {}
for row in ScrapedContribution.objects.values('donor', 'city', 'state', 'employer', 'occupation'):
    key = '-'.join([row[k] if row[k] else '' for k in row.keys()])
    data_dictionary[key] = {k: row[k] if row[k] else '' for k in row.keys()}

print data_dictionary.keys()[0]
print data_dictionary[data_dictionary.keys()[0]]
print len(data_dictionary)

sample = []
max_idx = 15000
for idx in range(max_idx):
    sample.append((data_dictionary[data_dictionary.keys()[idx]],
                   data_dictionary[data_dictionary.keys()[idx + max_idx]]))

-DIENNO LAURIE-WA--GIG HARBOR
{'donor': u'DIENNO LAURIE', 'state': u'WA', 'employer': '', 'city': u'GIG HARBOR', 'occupation': ''}
228521


In [8]:
deduper = Dedupe(variables, sample, num_cores=1)
print deduper

ZeroDivisionError: float division

In [ ]:
print [{
            'donor': donor['donor'],
            'city': donor['city'],
            'state': donor['state'],
            'employer': donor['employer'],
            'occupation': donor['occupation']
        } for donor in ScrapedContribution.objects.values('donor', 'city', 'state', 'employer', 'occupation')[:10]]

In [ ]:
for idx in range(10):
    print idx